# All notebooks in one

In [118]:
TIMES = dict()
RELATIVE_SPEEDUP = dict()
LOOPS_DEFAULT = 100/10
ITERATIONS_DEFAULT = 20000/10
NUMBER_DEFAULT = 1
REPEATS_DEFAULT = 1
CANARY_MODE_DEFAULT = False

In [9]:
"""
import timeit
def measure_file_old(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT, orig_globals=orig_globals):
    with open("nbody_%s.py" % file_suffix,'r') as f:
        total_statement = f.read()
    total_statement = total_statement.replace("nbody(100, 'sun', 20000)", "nbody(%i, 'sun', %i)" % (loops, iterations))
    total_statement = total_statement.replace("print", "")
    total_statement = "print('running')\n" + total_statement + "\n    print('done')" 
    with open("nbody_%s.canary.py" % file_suffix, 'w') as f:
        f.write(total_statement)
    return 999
    #times = [timeit.timeit(stmt=total_statement, number=3, globals=orig_globals) for it in range(number)]
    #return float(sum(times)) / float(len(times))
"""

'\nimport timeit\ndef measure_file_old(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT, orig_globals=orig_globals):\n    with open("nbody_%s.py" % file_suffix,\'r\') as f:\n        total_statement = f.read()\n    total_statement = total_statement.replace("nbody(100, \'sun\', 20000)", "nbody(%i, \'sun\', %i)" % (loops, iterations))\n    total_statement = total_statement.replace("print", "")\n    total_statement = "print(\'running\')\n" + total_statement + "\n    print(\'done\')" \n    with open("nbody_%s.canary.py" % file_suffix, \'w\') as f:\n        f.write(total_statement)\n    return 999\n    #times = [timeit.timeit(stmt=total_statement, number=3, globals=orig_globals) for it in range(number)]\n    #return float(sum(times)) / float(len(times))\n'

import timeit
import numpy as np
from subprocess import check_output

def measure_file(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT
                 , repeat=REPEATS_DEFAULT, canary_mode=CANARY_MODE_DEFAULT):
    with open("nbody_%s.py" % file_suffix,'r') as f:
        total_statement = f.read()
    
    
    total_statement = total_statement.replace("nbody(100, 'sun', 20000)", "nbody(%i, 'sun', %i)" % (loops, iterations))
    if canary_mode:
        total_statement = total_statement.replace("print", "")
        total_statement = "print('running')\n" + total_statement + "\n    print('done')" 
    
    
    with open("nbody_%s.canary.py" % file_suffix, 'w') as f:
        f.write(total_statement)
    
    times = timeit.repeat('check_output(["python", "nbody_%s.canary.py"])' % file_suffix,
                         setup='from subprocess import check_output',
                         number=number,
                         repeat=repeat)
    return np.array(times).mean() /number
        

In [138]:
import timeit
import numpy as np
from subprocess import check_output

def measure_file(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT
                 , repeat=REPEATS_DEFAULT, canary_mode=CANARY_MODE_DEFAULT, average=True):
    name = "nbody_%s" % file_suffix
    with open(name+".py",'r') as f:
        total_statement = f.read()
    
    if canary_mode:
        code = 'with redirect_stdout(open(os.devnull, "w")):\n\tnbody(%i,"sun",%i)' % (loops, iterations)
    else: 
        code = 'nbody(%i,"sun",%i)' % (loops, iterations)
        
    times = timeit.repeat(   code,
                             setup='import imp; import %s; imp.reload(%s); from %s import nbody; from contextlib import redirect_stdout; import os; nbody(1,"sun",1)' % (name, name, name),
                             number=number,
                             repeat=repeat)
    if average:
        return np.array(times).mean() /number
    else:
        return times

In [12]:
if False: 
    name = 'orig'
    TIMES[name] = measure_file(name)
    RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
    TIMES[name], RELATIVE_SPEEDUP[name]

## Original

In [5]:
%%file nbody_orig.py
"""
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Writing nbody_orig.py


In [6]:
!cp nbody.py nbody_orig.py

In [7]:
name = 'orig'
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(80.408898497000337, 1.0)

In [8]:
!rm nbody_orig.py

# Problem 1
Reducing function call overhead

In [9]:
%%file nbody_1.py
"""The biggest change was removing the duplicate computations of compute_b(),
which was recycled 3 times more than it should have.
Inisde of compute_b() was compute_m(), and that was being replicated 6 times.

I also got rid of the update_* functions, which helped just a little.

Also, removed the duplicate and nonfunctional report_energy()

"""
"""
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

#def compute_deltas(x1, x2, y1, y2, z1, z2):
#    return (x1-x2, y1-y2, z1-z2)
    
#def compute_b(m, dt, dx, dy, dz):
#    mag = compute_mag(dt, dx, dy, dz)
#    return m * mag

#def compute_mag(dt, dx, dy, dz):
#    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

#def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
#    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
#    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
#    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
#    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
#    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
#    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

#def update_rs(r, dt, vx, vy, vz):
#    r[0] += dt * vx
#    r[1] += dt * vy
#    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

                #update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                
                mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
                
                thing_for_v1 = m2 * mag
                v1[0] -= dx * thing_for_v1
                v1[1] -= dy * thing_for_v1
                v1[2] -= dz * thing_for_v1
                
                thing_for_v2 = m1 * mag            
                v2[0] += dx * thing_for_v2
                v2[1] += dy * thing_for_v2
                v2[2] += dz * thing_for_v2
                
                
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        #update_rs(r, dt, vx, vy, vz)
        r[0] += dt * vx
        r[1] += dt * vy
        r[2] += dt * vz

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

                #e -= compute_energy(m1, m2, dx, dy, dz)
                e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())


if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_1.py


In [10]:
name=1
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(30.994290845002979, 2.5943132204285995)

# Problem 2

using alternatices to membership testing of lists

In [11]:
%%file nbody_2.py

"""
This made very little difference.

The biggest change I made was iterating through the body pairs using itertools.combinations( <body_name_list>, 2).

It also helped to store this result as an available variable.
"""

"""
    N-body simulation.
"""
from itertools import combinations

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}
VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    
    for (body1, body2) in VISIT_SCHEDULE:
        ([x1, y1, z1], v1, m1) = BODIES[body1]
        ([x2, y2, z2], v2, m2) = BODIES[body2]
        (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
                    
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]
        (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        
        e -= compute_energy(m1, m2, dx, dy, dz)
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy()
        for _ in range(iterations):
            advance(0.01)
        print(report_energy())

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_2.py


In [12]:
name = 2
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(69.164767191978171, 1.162570218357162)

# Problem 3

using local rather than global variables

In [13]:
%%file nbody_3.py
"""

I gave all functions references to the varaible BODIES which I used several times.  
However, this didn't help very much at all.

    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt, BODIES=BODIES):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)


def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0, BODIES=BODIES):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    for _ in range(loops):
        report_energy(BODIES=BODIES)
        for _ in range(iterations):
            advance(0.01, BODIES=BODIES)
        print(report_energy(BODIES=BODIES))


if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_3.py


In [14]:
name = 3
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(78.692762500955723, 1.0218080537714478)

In [15]:
A = 1

def f(A=A):
    A += 1
    print("f = %s" % A)
    
def g(A=A):
    A += 1
    print("g = %s" % A)
    
f()
g()

f = 2
g = 2


In [16]:
A = dict()

def f(A=A):
    A['a'] = 'a'
    print("f = %s" % A)
    
def g(A=A):
    A['b'] = 'b'
    print("g = %s" % A)
    
f()
g()

f = {'a': 'a'}
g = {'b': 'b', 'a': 'a'}


# Problem 4
using data aggregation ro reduce loop overhead

In [56]:
%%file nbody_4.py
"""
    The biggest change here, by far, was using map() for the inner and outer loop of nbody()
    
    I also changed most of the other for loops to use map() instead.  This made only marginal differences.
    
    N-body simulation.
"""

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

def compute_deltas(x1, x2, y1, y2, z1, z2):
    return (x1-x2, y1-y2, z1-z2)
    
def compute_b(m, dt, dx, dy, dz):
    mag = compute_mag(dt, dx, dy, dz)
    return m * mag

def compute_mag(dt, dx, dy, dz):
    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

def update_rs(r, dt, vx, vy, vz):
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz

def advance(dt):
    '''
        advance the system one timestep
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ([x1, y1, z1], v1, m1) = BODIES[body1]
                ([x2, y2, z2], v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                update_vs(v1, v2, dt, dx, dy, dz, m1, m2)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        update_rs(r, dt, vx, vy, vz)

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0):
    '''
        compute the energy and return it so that it can be printed
    '''
    seenit = []
    for body1 in BODIES.keys():
        for body2 in BODIES.keys():
            if (body1 != body2) and not (body2 in seenit):
                ((x1, y1, z1), v1, m1) = BODIES[body1]
                ((x2, y2, z2), v2, m2) = BODIES[body2]
                (dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
                e -= compute_energy(m1, m2, dx, dy, dz)
                seenit.append(body1)
        
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES.keys():
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    def i_func(_):
        advance(0.01)
    
    def l_func(_):
        report_energy()
        list(map(i_func, range(iterations)))
        print(report_energy())
                                    
                                    
    list(map(l_func, range(loops)))

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_4.py


In [57]:
name = 4
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(76.967735138954595, 1.0447091674431266)

# opt

In [51]:
%%file nbody_opt.py
"""
    By far, the biggest change was removing the duplicated function calls.
    
    I tried parallelization for the loops where it was safe to do them, but it was actually slower.     
     
    N-body simulation.
"""
from itertools import combinations


PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

#def compute_deltas(x1, x2, y1, y2, z1, z2):
#    return (x1-x2, y1-y2, z1-z2)
    
#def compute_b(m, dt, dx, dy, dz):
#    mag = compute_mag(dt, dx, dy, dz)
#    return m * mag

#def compute_mag(dt, dx, dy, dz):
#    return dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

#def update_vs(v1, v2, dt, dx, dy, dz, m1, m2):
#    v1[0] -= dx * compute_b(m2, dt, dx, dy, dz)
#    v1[1] -= dy * compute_b(m2, dt, dx, dy, dz)
#    v1[2] -= dz * compute_b(m2, dt, dx, dy, dz)
#    v2[0] += dx * compute_b(m1, dt, dx, dy, dz)
#    v2[1] += dy * compute_b(m1, dt, dx, dy, dz)
#    v2[2] += dz * compute_b(m1, dt, dx, dy, dz)

#def update_rs(r, dt, vx, vy, vz):
#    r[0] += dt * vx
#    r[1] += dt * vy
#    r[2] += dt * vz


def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    thing_for_v1 = m2 * mag
    v1[0] -= dx * thing_for_v1
    v1[1] -= dy * thing_for_v1
    v1[2] -= dz * thing_for_v1

    thing_for_v2 = m1 * mag            
    v2[0] += dx * thing_for_v2
    v2[1] += dy * thing_for_v2
    v2[2] += dz * thing_for_v2

def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    #update_rs(r, dt, vx, vy, vz)
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
def advance(dt=dt, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        advance the system one timestep
    '''
    
    list(map(to_do, BODIES1, BODIES2))      
    list(map(update_rs_for_body, BODIES_KEYS))
    

def compute_energy(m1, m2, dx, dy, dz):
    return (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]
        #(dx, dy, dz) = compute_deltas(x1, x2, y1, y2, z1, z2)
        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        #e -= compute_energy(m1, m2, dx, dy, dz)
        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    #for body, (r, [vx, vy, vz], m) in BODIES.items():
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

        
    for _ in range(loops):
        for _ in range(iterations):            
            list(map(to_do, BODIES1, BODIES2))      
            list(map(update_rs_for_body, BODIES_KEYS))
            #advance(0.01, BODIES=BODIES)
        print(report_energy(BODIES=BODIES))
            
    """    
    def i_func(_):
        advance()
    
    def l_func(_):
        list(map(i_func, range(iterations)))
        print(report_energy())
                                    
                                    
    list(map(l_func, range(loops)))
"""

if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_opt.py


In [45]:
name = 'opt'
TIMES[name] = measure_file(name)
RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
TIMES[name], RELATIVE_SPEEDUP[name]

(28.600031132984441, 2.8114968869479546)

In [46]:
RELATIVE_SPEEDUP

{1: 2.5943132204285995,
 2: 1.162570218357162,
 3: 1.0218080537714478,
 4: 0.99983733795216001,
 'orig': 1.0,
 'opt': 2.8114968869479546}

In [47]:
did_i_break_30 = TIMES['opt'] < 30

did_i_break_30 = "I did break 30 seconds. :)" if did_i_break_30 else "I did not break 30 seconds. :("
did_i_break_30

'I did break 30 seconds. :)'

In [52]:
with open("nbody_opt.py", 'r') as f:
    text = f.read()
    
with open("nbody_opt.py", 'w') as f:
    text = ("""\n 
    \"\"\" 
    
    The relative speedups are: 
    %s
    
    The time it took to run is:
    %s
    
    %s
    
    \"\"\"\n
    """ % (RELATIVE_SPEEDUP, TIMES, did_i_break_30)
           ) + text
    
    f.write(text)

In [53]:
TIMES

{1: 30.994290845002979,
 2: 69.164767191978171,
 3: 78.692762500955723,
 4: 80.421980100974906,
 'orig': 80.408898497000337,
 'opt': 28.600031132984441}

In [50]:
!rm *.canary.py

In [13]:
name="iter"
TIMES[name] = measure_file(name)

In [14]:
TIMES

{'iter': 3.1431850459193811}

# Assignment 6

In [74]:
%%file nbody_numpy.py
"""
For assignment6, I got a speedup of 54%.
"""
 
""" 
    
    The relative speedups are: 
    {1: 2.5943132204285995, 2: 1.162570218357162, 3: 1.0218080537714478, 4: 0.99983733795216001, 'orig': 1.0, 'opt': 2.8114968869479546}
    
    The time it took to run is:
    {1: 30.994290845002979, 2: 69.164767191978171, 3: 78.692762500955723, 4: 80.421980100974906, 'orig': 80.408898497000337, 'opt': 28.600031132984441}
    
    I did break 30 seconds. :)
    
"""

"""
    By far, the biggest change was removing the duplicated function calls.
    
    I tried parallelization for the loops where it was safe to do them, but it was actually slower.     
     
    N-body simulation.
"""
import itertools
import numpy as np

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': (np.array([0.0, 0.0, 0.0]), np.array([0.0, 0.0, 0.0]), SOLAR_MASS),

    'jupiter': (np.array([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01]),
                np.array([1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR]),
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': (np.array([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01]),
               np.array([-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR]),
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': (np.array([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01]),
               np.array([2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR]),
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': (np.array([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01]),
                np.array([2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR]),
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    (xyz1, v1, m1) = BODIES[body1]
    (xyz2, v2, m2) = BODIES[body2]
    d = xyz1-xyz2
    
    mag = dt * (np.inner(d,d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1 = d * body1_velocity_arg


    body2_velocity_arg = m1 * mag       
    v2 = d * body1_velocity_arg


def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, vxyz, m) = BODIES[body]
    r = dt * vxyz

    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        (xyz1, v1, m1) = BODIES[body1]
        (xyz2, v2, m2) = BODIES[body2]

        d = xyz1 - xyz2

        # Amazingly the code as written is faster than this code below.
        #e -= (m1 * m2) / np.inner(d,d)**0.5
        e -= (m1 * m2) / (d[0]**2 + d[1]**2 + d[2]**2)**0.5

        
    for body in BODIES_KEYS:
        (r, v, m) = BODIES[body]
        e += m * np.inner(v,v) / 2.
        
    return e

def offset_momentum(ref, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.zeros(3)
    for body in BODIES_KEYS:
        (r, v, m) = BODIES[body]
        p -= v*m
        
    (r, v, m) = ref
    v = p / m


def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            print(report_energy())
            
        if i >= total_runs - 1:
            break
      


if __name__ == '__main__':
    nbody(100, 'sun', 20000)



Overwriting nbody_numpy.py


In [69]:
%lprun -f nbody nbody(100, 'sun', 200)

152253.84139430564
152253.84127495112
152253.84116736974
152253.84105123498
152253.84094655924
152253.8408334889
152253.8407315785
152253.84062142877
152253.8405221536
152253.84041479154
152253.84031803143
152253.84021333396
152253.84011897785
152253.8400168301
152253.83992477413
152253.8398250694
152253.83973521722
152253.83963785486
152253.83955011555
152253.8394550023
152253.83936929164
152253.83927633957
152253.8391925781
152253.83910170454
152253.83901981765
152253.8389309451
152253.83885086284
152253.83876391748
152253.83868557314
152253.83860048605
152253.83852381748
152253.8384405232
152253.83836547125
152253.83828390797
152253.83821041693
152253.83813052555
152253.83805854205
152253.83798026666
152253.8379097402
152253.83783302788
152253.8377639108
152253.83768871098
152253.83762095755
152253.83754722247
152253.83748078966
152253.83740847337
152253.83734331955
152253.83727237768
152253.83720846297
152253.83713885426
152253.83707614138
152253.83700782605
152253.83694627922
1522

In [22]:
import line_profiler

In [ ]:
BEFORE
Timer unit: 1e-06 s

Total time: 0.174554 s
File: <ipython-input-62-72746bac4545>
Function: nbody at line 135

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   135                                           def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
   136                                               '''
   137                                                   nbody simulation
   138                                                   loops - number of loops to run
   139                                                   reference - body at center of system
   140                                                   iterations - number of timesteps to advance
   141                                               '''
   142                                               # Set up global state
   143         1           15     15.0      0.0      offset_momentum(BODIES[reference])
   144                                           
   145         1            1      1.0      0.0      total_runs = loops * iterations
   146     20000        10542      0.5      6.0      for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
   147     20000        95873      4.8     54.9          to_do(body1, body2)
   148                                                   
   149     20000        10791      0.5      6.2          if (i % iterations) == 0:
   150       600          279      0.5      0.2              for body in BODIES_KEYS:
   151       500          887      1.8      0.5                  update_rs_for_body(body)
   152                                                           
   153     20000        10496      0.5      6.0          if (i % loops) == 0:
   154       200        35436    177.2     20.3              print(report_energy())
   155                                                       
   156     20000        10233      0.5      5.9          if i >= total_runs - 1:
   157         1            1      1.0      0.0              break

In [23]:
%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [61]:
%lprun -f offset_momentum nbody(100, 'sun', 200)

-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.169289903378
-0.16928

In [24]:
%lprun -f nbody -f offset_momentum -f report_energy -f update_rs_for_body -f to_do nbody(100, 'sun', 200)

-0.168516264829
-0.166736555555
-0.164287193043
-0.160619886008
-0.156468727887
-0.150895251389
-0.145023737691
-0.137516187648
-0.129897312672
-0.120418079527
-0.11101603672
-0.0995172962305
-0.0882869991163
-0.0747100535494
-0.0615965200732
-0.0458709720065
-0.0308085557
-0.0128512914257
0.00423726198674
0.0245233086734
0.0437280063409
0.0664553468526
0.0878803297249
0.113178716296
0.136943918898
0.164962501487
0.19120564461
0.22211556271
0.250994547212
0.284992049757
0.316687837772
0.353998049691
0.388718143094
0.429599630258
0.467582287024
0.512332615166
0.553851984985
0.602814526712
0.648186941602
0.701759264488
0.751350993504
0.809995269778
0.864232147015
0.928488173692
0.987867646895
1.05836927229
1.12347561251
1.20097165832
1.27249534459
1.35787653386
1.4366392204
1.53097323655
1.6179602846
1.72253799949
1.8189414108
1.93533870201
2.0426145861
2.17277629971
2.29272301232
2.43907900765
2.57394527516
2.73957397671
2.89221148961
3.08107554682
3.25515916117
3.47245366871
3.67280734

In [20]:
import cProfile
cProfile.run("nbody(100, 'sun', 2000)")

167.318821214
167.31602507
167.313230714
167.310438146
167.307647365
167.304858372
167.302071167
167.29928575
167.296502121
167.293720279
167.290940225
167.288161959
167.285385481
167.28261079
167.279837887
167.277066773
167.274297445
167.271529906
167.268764155
167.266000191
167.263511534
167.26080308
167.25809637
167.255391403
167.252688179
167.249986699
167.247286962
167.244588969
167.241892719
167.239198212
167.236505449
167.233814429
167.231125152
167.228437619
167.225751829
167.223067782
167.220385479
167.217704919
167.215026103
167.212349029
167.209938647
167.207314797
167.204692649
167.202072203
167.199453459
167.196836418
167.194221079
167.191607442
167.188995507
167.186385275
167.183776745
167.181169917
167.178564791
167.175961367
167.173359646
167.170759627
167.16816131
167.165564695
167.162969782
167.160376572
167.158041734
167.155499618
167.152959166
167.150420378
167.147883255
167.145347796
167.142814001
167.14028187
167.137751403
167.1352226
167.132695461
167.130169987
1

In [ ]:

Total time: 0.218493 s
File: <ipython-input-21-d08a09d75a7a>
Function: to_do at line 69

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    69                                           def to_do(body1, body2, BODIES=BODIES, dt=dt):
    70     20000        29288      1.5     13.4      ([x1, y1, z1], v1, m1) = BODIES[body1]
    71     20000        27295      1.4     12.5      ([x2, y2, z2], v2, m2) = BODIES[body2]
    72     20000        15784      0.8      7.2      (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    73                                               
    74     20000        21118      1.1      9.7      mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
    75                                           
    76     20000        11350      0.6      5.2      body1_velocity_arg = m2 * mag
    77     20000        17830      0.9      8.2      v1[0] -= dx * body1_velocity_arg
    78     20000        16458      0.8      7.5      v1[1] -= dy * body1_velocity_arg
    79     20000        16508      0.8      7.6      v1[2] -= dz * body1_velocity_arg
    80                                           
    81     20000        11412      0.6      5.2      body2_velocity_arg = m1 * mag            
    82     20000        18134      0.9      8.3      v2[0] += dx * body2_velocity_arg
    83     20000        16797      0.8      7.7      v2[1] += dy * body2_velocity_arg
    84     20000        16519      0.8      7.6      v2[2] += dz * body2_velocity_arg

In [ ]:
Timer unit: 1e-06 s

Total time: 0.192647 s
File: <ipython-input-27-136a5cb84e62>
Function: to_do at line 69

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    69                                           def to_do(body1, body2, BODIES=BODIES, dt=dt):
    70     20000        11162      0.6      5.8      (xyz1, v1, m1) = BODIES[body1]
    71     20000        11447      0.6      5.9      (xyz2, v2, m2) = BODIES[body2]
    72     20000        19750      1.0     10.3      d = xyz1-xyz2
    73                                               
    74     20000        78437      3.9     40.7      mag = dt * ((d**2).sum() ** (-1.5))
    75                                           
    76     20000        10986      0.5      5.7      body1_velocity_arg = m2 * mag
    77     20000        27016      1.4     14.0      v1 = d * body1_velocity_arg
    78                                               #v1[0] -= dx * body1_velocity_arg
    79                                               #v1[1] -= dy * body1_velocity_arg
    80                                               #v1[2] -= dz * body1_velocity_arg
    81                                           
    82     20000        10670      0.5      5.5      body2_velocity_arg = m1 * mag       
    83     20000        23179      1.2     12.0      v2 = d * body1_velocity_arg

In [ ]:
Timer unit: 1e-06 s

Total time: 0.187241 s
File: <ipython-input-34-367ef479bb0a>
Function: to_do at line 69

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    69                                           def to_do(body1, body2, BODIES=BODIES, dt=dt):
    70     20000        11139      0.6      5.9      (xyz1, v1, m1) = BODIES[body1]
    71     20000        10533      0.5      5.6      (xyz2, v2, m2) = BODIES[body2]
    72     20000        20230      1.0     10.8      d = xyz1-xyz2
    73                                               
    74     20000        67103      3.4     35.8      mag = dt * (np.inner(d,d) ** (-1.5))
    75                                           
    76     20000        12890      0.6      6.9      body1_velocity_arg = m2 * mag
    77     20000        29594      1.5     15.8      v1 = d * body1_velocity_arg
    78                                               #v1[0] -= dx * body1_velocity_arg
    79                                               #v1[1] -= dy * body1_velocity_arg
    80                                               #v1[2] -= dz * body1_velocity_arg
    81                                           
    82     20000        11343      0.6      6.1      body2_velocity_arg = m1 * mag       
    83     20000        24409      1.2     13.0      v2 = d * body1_velocity_arg

In [31]:
from numpy import linalg as LA
LA.norm([1,2], 2)

2.2360679774997898

In [33]:
a = np.random.random(3)
%timeit np.inner(a, a)
%timeit np.sum(a**2)


The slowest run took 14.83 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 934 ns per loop
The slowest run took 10.85 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.56 µs per loop


In [ ]:
Timer unit: 1e-06 s

Total time: 0.002361 s
File: <ipython-input-36-0ac954484f85>
Function: update_rs_for_body at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    84                                           def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    85       500         1090      2.2     46.2      (r, [vx, vy, vz], m) = BODIES[body]
    86       500          544      1.1     23.0      r[0] += dt * vx
    87       500          370      0.7     15.7      r[1] += dt * vy
    88       500          357      0.7     15.1      r[2] += dt * vz

In [ ]:


Total time: 0.016553 s
File: <ipython-input-43-b3a49c1fe5b3>
Function: report_energy at line 89

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    89                                           def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    90                                               '''
    91                                                   compute the energy and return it so that it can be printed
    92                                               '''
    93      2200         1472      0.7      8.9      for (body1, body2) in VISIT_SCHEDULE:
    94      2000         3688      1.8     22.3          ((x1, y1, z1), v1, m1) = BODIES[body1]
    95      2000         3001      1.5     18.1          ((x2, y2, z2), v2, m2) = BODIES[body2]
    96                                           
    97      2000         1774      0.9     10.7          (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    98                                           
    99      2000         2802      1.4     16.9          e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
   100                                                   
   101      1200          753      0.6      4.5      for body in BODIES_KEYS:
   102      1000         1578      1.6      9.5          (r, [vx, vy, vz], m) = BODIES[body]
   103      1000         1366      1.4      8.3          e += m * (vx * vx + vy * vy + vz * vz) / 2.
   104                                                   
   105       200          119      0.6      0.7      return e

In [ ]:
Timer unit: 1e-06 s

Total time: 0.018095 s
File: <ipython-input-47-3c94c6f28c27>
Function: report_energy at line 89

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
    89                                           def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    90                                               '''
    91                                                   compute the energy and return it so that it can be printed
    92                                               '''
    93      2200         1506      0.7      8.3      for (body1, body2) in VISIT_SCHEDULE:
    94      2000         1227      0.6      6.8          (xyz1, v1, m1) = BODIES[body1]
    95      2000         1105      0.6      6.1          (xyz2, v2, m2) = BODIES[body2]
    96                                           
    97      2000         2154      1.1     11.9          d = xyz1 - xyz2
    98                                           
    99      2000         7901      4.0     43.7          e -= (m1 * m2) / np.sqrt(np.inner(d,d))
   100                                                   
   101      1200          705      0.6      3.9      for body in BODIES_KEYS:
   102      1000          582      0.6      3.2          (r, v, m) = BODIES[body]
   103      1000         2818      2.8     15.6          e += m * np.inner(v,v) / 2.
   104                                                   
   105       200           97      0.5      0.5      return e

# Assignment 8

Use your knowledge of Numba to convert the nbody_opt.py program you wrote in Assignment 3 into a Numba program. Pay particular attention to the following:

Add @jit decorators to all funcitons
Add function signatures to all funcitons
Place the resulting code in a file called nbody_numba.py and check that it produces the same results as the original program.

When you are satisfied that your program is working, commit it to the same repository you used in Assignment 3

In [6]:
%%file nbody_numba_baseline.py
"""

This was actually based off of nbody_iter.py, beacuse I didn't like my nbody_opt.py code


"""

from numba import jit
import itertools

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[0] -= dx * body1_velocity_arg
    v1[1] -= dy * body1_velocity_arg
    v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag            
    v2[0] += dx * body2_velocity_arg
    v2[1] += dy * body2_velocity_arg
    v2[2] += dz * body2_velocity_arg


def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            print(report_energy())
            
        if i >= total_runs - 1:
            break
      


if __name__ == '__main__':
    nbody(100, 'sun', 20000)


Writing nbody_numba_baseline.py


In [120]:
name = 'numba_baseline'
TIMES[name] = measure_file(name, canary_mode=True)
TIMES
#RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
#TIMES[name], RELATIVE_SPEEDUP[name]

{'numba_baseline': 0.043400720998761244}

In [136]:
%%file nbody_numba_justjit.py
"""

This was actually based off of nbody_iter.py, beacuse I didn't like my nbody_opt.py code


"""

from numba import jit
import itertools

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

@jit
def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[0] -= dx * body1_velocity_arg
    v1[1] -= dy * body1_velocity_arg
    v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag            
    v2[0] += dx * body2_velocity_arg
    v2[1] += dy * body2_velocity_arg
    v2[2] += dz * body2_velocity_arg

@jit('void(float64, float64[:,:,:], char[:,:], char[:], int32)')
def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    

@jit   
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

@jit
def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m

@jit
def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            #print(report_energy())
            pass
            
        if i >= total_runs - 1:
            break
      
if __name__ == '__main__':
    nbody(100, 'sun', 20000)


Overwriting nbody_numba_justjit.py


In [143]:
import timeit
import numpy as np
from subprocess import check_output

def measure_file(file_suffix, number=NUMBER_DEFAULT, loops=LOOPS_DEFAULT, iterations = ITERATIONS_DEFAULT
                 , repeat=REPEATS_DEFAULT, canary_mode=CANARY_MODE_DEFAULT, average=True):
    name = "nbody_%s" % file_suffix
    
    if canary_mode:
        code = 'with redirect_stdout(open(os.devnull, "w")):\n\tnbody(%i,"sun",%i)' % (loops, iterations)
    else: 
        code = 'nbody(%i,"sun",%i)' % (loops, iterations)
        
    times = timeit.repeat(   code,
                             setup='import imp; import %s; imp.reload(%s); from %s import nbody; from contextlib import redirect_stdout; import os;' % (name, name, name),
                             number=number,
                             repeat=repeat)
    if average:
        return np.array(times).mean() /number
    else:
        return times

In [144]:
name = 'numba_justjit'
TIMES[name] = measure_file(name, canary_mode=True, repeat=3, number=1, average=False)
TIMES
#RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
#TIMES[name], RELATIVE_SPEEDUP[name]

{'numba_baseline': 0.043400720998761244,
 'numba_justjit': [4.309290709999914, 4.2915096010001434, 4.370692231001158]}

In [ ]:
%%file nbody_numba_justjit.py
"""

This was actually based off of nbody_iter.py, beacuse I didn't like my nbody_opt.py code


"""

from numba import jit
import itertools

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01

@jit
def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[0] -= dx * body1_velocity_arg
    v1[1] -= dy * body1_velocity_arg
    v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag            
    v2[0] += dx * body2_velocity_arg
    v2[1] += dy * body2_velocity_arg
    v2[2] += dz * body2_velocity_arg

@jit
def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    

@jit   
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

@jit
def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m

@jit
def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            #print(report_energy())
            pass
            
        if i >= total_runs - 1:
            break
      
if __name__ == '__main__':
    nbody(100, 'sun', 20000)


In [270]:
#%%file nbody_numba.py
"""

This was actually based off of nbody_iter.py, beacuse I didn't like my nbody_opt.py code


"""

from numba import jit
import numpy as np
import itertools

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(itertools.combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
dt = 0.01


# alter data types to arrays
# BODIES IS NOW SPLIT INTO BODIES1_XYZ, BODIES2_V, BODIES2_MASS
BODIES_2_XYZ = np.array([BODIES[k][0] for k in BODIES_KEYS], dtype="float64")
BODIES_2_V = np.array([BODIES[k][1] for k in BODIES_KEYS], dtype="float64")
BODIES_2_MASS = np.array([BODIES[k][2] for k in BODIES_KEYS], dtype="float64")

BODIES_KEYS = np.array( [i for i, k in enumerate(BODIES_KEYS)], dtype="int64")
VISIT_SCHEDULE = np.array(list(itertools.combinations(BODIES_KEYS, 2)), dtype='int64')
BODIES1, BODIES2= (np.array(it, dtype='int64') for it in zip(*VISIT_SCHEDULE))

@jit("Tuple((float64[:], float64[:], float64))(int64, float64[:,:], float64[:,:], float64[:])")
def get_from_bodies(body, BODIES_2_XYZ=BODIES_2_XYZ, BODIES_2_V=BODIES_2_V, BODIES_2_MASS = BODIES_2_MASS):
    return (BODIES_2_XYZ[body], BODIES_2_V[body], BODIES_2_MASS[body])

@jit("void(int64, int64, float64)")
def to_do(body1, body2, dt=dt):
    (xyz1, v1, m1) = get_from_bodies(body1)
    (xyz2, v2, m2) = get_from_bodies(body2)
    d = xyz1 - xyz2
    
    mag = dt * (np.sum(d*d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1 = v1 - d * body1_velocity_arg
    #v1[0] -= dx * body1_velocity_arg
    #v1[1] -= dy * body1_velocity_arg
    #v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag   
    v2 = v2 - d * body2_velocity_arg
    #v2[0] += dx * body2_velocity_arg
    #v2[1] += dy * body2_velocity_arg
    #v2[2] += dz * body2_velocity_arg

@jit("int64(int64, int64)")
def update_rs_for_body(body, dt=dt):
    (r, [vx, vy, vz], m) = get_from_bodies(body)
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
@jit("int64(int64[:])")
def report_energy(BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = 0.0
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = get_from_bodies(body1)
        ((x2, y2, z2), v2, m2) = get_from_bodies(body2)

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, v, m) = get_from_bodies(body)
        e += m * np.sum(v*v) / 2.
        
    return e

@jit("void(int64, int64[:])")
def offset_momentum(ref, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.array([0,0,0], dtype='float64')
                         
    for body in BODIES_KEYS:
        (r, v, m) = get_from_bodies(body)
        p = p - v * m
        #px -= vx * m
        #py -= vy * m
        #pz -= vz * m
        
    (r, v, m) = get_from_bodies(ref)
    v = p / m
    #v[0] = px / m
    #v[1] = py / m
    #v[2] = pz / m

def nbody(loops, reference, iterations, VISIT_SCHEDULE=VISIT_SCHEDULE,
          BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    reference = 0
    # Set up global state
    offset_momentum(reference)

    total_runs = loops * iterations
    for i, (body1, body2) in enumerate(itertools.cycle(VISIT_SCHEDULE)):
        to_do(body1, body2)
        
        if (i % iterations) == 0:
            for body in BODIES_KEYS:
                update_rs_for_body(body)
                
        if (i % loops) == 0:
            print(report_energy())
            pass
            
        if i >= total_runs - 1:
            break

            
#if __name__ == '__main__':
#    nbody(10, 'sun', 2000)

@jit(numba.int64(numba.int64, \
                 numba.types.Omitted(BODIES_2_XYZ), \
                 numba.types.Omitted(BODIES_2_V), \
                 numba.types.Omitted(BODIES_2_MASS) \
                ) \
    )
def get_from_bodies2(body, 
                    BODIES_2_XYZ=BODIES_2_XYZ,
                    BODIES_2_V=BODIES_2_V,
                    BODIES_2_MASS = BODIES_2_MASS):
    return 1
    return (BODIES_2_XYZ[body], BODIES_2_V[body], BODIES_2_MASS[body])
get_from_bodies2(1)#, BODIES_2_XYZ, BODIES_2_V, BODIES_2_MASS)

TypeError: No matching definition for argument type(s) int64, omitted(default=array([[  0.        ,   0.        ,   0.        ],
       [ 12.89436956, -15.1111514 ,  -0.22330758],
       [  8.34336672,   4.12479856,  -0.40352342],
       [ 15.37969711, -25.91931461,   0.17925877],
       [  4.84143144,  -1.16032004,  -0.10362204]])), omitted(default=array([[ 0.        ,  0.        ,  0.        ],
       [ 1.08279101,  0.86871302, -0.01083264],
       [-1.01077435,  1.82566237,  0.00841576],
       [ 0.97909073,  0.594699  , -0.03475596],
       [ 0.60632639,  2.81198684, -0.02521836]])), omitted(default=array([  3.94784176e+01,   1.72372406e-03,   1.12863261e-02,
         2.03368687e-03,   3.76936749e-02]))

In [273]:

@jit(numba.int64(numba.int64, \
                 numba.types.Omitted(BODIES_2_XYZ), \
                ) \
    )
def get_from_bodies2(body, 
                    BODIES_2_XYZ=BODIES_2_XYZ
                    ):
    
    return 1
get_from_bodies2(1)#, BODIES_2_XYZ, BODIES_2_V, BODIES_2_MASS)

TypeError: No matching definition for argument type(s) int64, omitted(default=array([[  0.        ,   0.        ,   0.        ],
       [ 12.89436956, -15.1111514 ,  -0.22330758],
       [  8.34336672,   4.12479856,  -0.40352342],
       [ 15.37969711, -25.91931461,   0.17925877],
       [  4.84143144,  -1.16032004,  -0.10362204]]))

In [209]:
name = 'numba'
TIMES[name] = measure_file(name, canary_mode=True, repeat=3, number=1, average=False)
TIMES
#RELATIVE_SPEEDUP[name] = TIMES['orig'] / TIMES[name]
#TIMES[name], RELATIVE_SPEEDUP[name]

SyntaxError: unexpected EOF while parsing (<string>, line 1)

In [226]:
numba.typeof(BODIES_2_V)

array(float64, 2d, C)

In [199]:
numba.typeof( (np.array(1), np.array(1), 1 ))

(array(int64, 0d, C), array(int64, 0d, C), int64)

In [237]:
numba.types.Omitted(numba.float64[:])

omitted(default=array(float64, 1d, A))

In [266]:
import numba
@jit(numba.float64(numba.float64, numba.types.Omitted([1,2,3])))
def f(b, a=5):
    return a

f(1)

5

In [262]:
numba.types.Omitted([1,2,3])

omitted(default=[1, 2, 3])